In [46]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
#from fuzzywuzzy import fuzz
import pickle

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
%matplotlib inline

In [47]:
with open('title.pkl', 'rb') as f:
    book_title = pickle.load(f)

In [48]:
with open('rating.pkl', 'rb') as f:
    book_rating = pickle.load(f)

In [49]:
with open('number_rating.pkl', 'rb') as f:
    book_number_rating = pickle.load(f)

In [50]:
with open('year_published.pkl', 'rb') as f:
    book_year_published = pickle.load(f)

In [51]:
with open('movie_data.pkl', 'rb') as f:
    movie_data = pickle.load(f)

In [52]:
book_data = pd.DataFrame(
    {'book_title': book_title,
     'book_rating': book_rating,
     'book_number_rating': book_number_rating,
     'book_year_published': book_year_published
    })

In [53]:
with open('book_data.pkl', 'wb') as f:
    pickle.dump(book_data, f)

In [54]:
book_data.dtypes

book_title             object
book_rating            object
book_number_rating     object
book_year_published    object
dtype: object

In [55]:
book_data["book_rating"] = pd.to_numeric(book_data["book_rating"])

In [56]:
book_data.dtypes

book_title              object
book_rating            float64
book_number_rating      object
book_year_published     object
dtype: object

In [57]:
movie_data.dtypes

adult                     object
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [58]:
book_movie = pd.merge(book_data, movie_data,  left_on='book_title', right_on = 'title')

In [59]:
book_movie.shape

(705, 28)

In [60]:
book_movie.dtypes

book_title                object
book_rating              float64
book_number_rating        object
book_year_published       object
adult                     object
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [61]:
book_movie.corr()

,book_rating,budget,revenue,runtime,vote_average,vote_count
book_rating,1.000000,0.119831,0.171653,-0.005385,0.206703,0.202751
budget,0.119831,1.000000,0.753144,0.121404,0.099657,0.603412
revenue,0.171653,0.753144,1.000000,0.113938,0.158091,0.747391
runtime,-0.005385,0.121404,0.113938,1.000000,0.001977,0.109464
vote_average,0.206703,0.099657,0.158091,0.001977,1.000000,0.272046
vote_count,0.202751,0.603412,0.747391,0.109464,0.272046,1.000000


In [63]:
book_movie.sample(5)

,book_title,book_rating,book_number_rating,book_year_published,adult,belongs_to_collection,budget,genres,homepage,id,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
247,The Client,3.99,"357,034",1993,False,NaN,45000000,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,10731,...,1994-07-20,117615211.0,119.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A district attorney out for a conviction. A ne...,The Client,False,6.4,287.0
504,Addicted,4.26,"16,452",1998,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,235271,...,2014-10-10,0.0,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Every Woman Needs an Escape,Addicted,False,5.4,63.0
693,Three Men in a Boat,3.89,"47,431",1889,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,252746,...,1956-12-23,0.0,84.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Three Men in a Boat,False,6.5,2.0
390,I Am Legend,4.06,"76,796",1954,False,NaN,150000000,"[{'id': 18, 'name': 'Drama'}, {'id': 27, 'name...",http://iamlegend.warnerbros.com/,6479,...,2007-12-14,585349010.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The last man on Earth is not alone,I Am Legend,False,6.9,4977.0
505,The Dinner,3.21,"115,446",2009,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 53, 'name...",NaN,401546,...,2017-05-05,1323312.0,120.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,How far would you go to protect your children?,The Dinner,False,4.1,44.0


In [66]:
book_movie.corr()

,book_rating,budget,revenue,runtime,vote_average,vote_count
book_rating,1.000000,0.119831,0.171653,-0.005385,0.206703,0.202751
budget,0.119831,1.000000,0.753144,0.121404,0.099657,0.603412
revenue,0.171653,0.753144,1.000000,0.113938,0.158091,0.747391
runtime,-0.005385,0.121404,0.113938,1.000000,0.001977,0.109464
vote_average,0.206703,0.099657,0.158091,0.001977,1.000000,0.272046
vote_count,0.202751,0.603412,0.747391,0.109464,0.272046,1.000000


In [67]:
with open('book_movie_data.pkl', 'wb') as f:
    pickle.dump(book_movie, f)